In [1]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import Imputer
from sklearn.metrics import accuracy_score
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import zscore
from sklearn.preprocessing import Imputer
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [2]:
data=pd.read_excel("loantrain.xlsx")
data.head(10)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LID001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LID001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LID001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LID001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LID001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LID001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LID001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LID001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LID001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LID001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [3]:
data.shape

(614, 13)

In [4]:
data.dtypes

Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object

In [5]:
data.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [6]:
data.Gender.value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [7]:
pd.crosstab(data.Gender, data.Married)

Married,No,Yes
Gender,,
Female,80,31
Male,130,357


In [8]:
pd.crosstab(data.Gender, data.Education)

Education,Graduate,Not Graduate
Gender,,
Female,92,20
Male,376,113


In [9]:
data.Gender.fillna("Male", inplace=True)

In [10]:
data[data.Gender.isnull()]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status


In [11]:
data.Dependents.value_counts()

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [12]:
pd.crosstab(data.Gender, data.Dependents)

Dependents,0,1,2,3+
Gender,,,,
Female,80,19,7,3
Male,265,83,94,48


In [13]:
pd.crosstab(data.Education,data.Dependents)

Dependents,0,1,2,3+
Education,,,,
Graduate,275,81,77,36
Not Graduate,70,21,24,15


In [14]:
pd.crosstab(data.Property_Area,data.Dependents)

Dependents,0,1,2,3+
Property_Area,,,,
Rural,109,21,29,18
Semiurban,130,40,37,20
Urban,106,41,35,13


In [15]:
data.Dependents.fillna(0, inplace=True)

In [16]:
data.isnull().sum()

Loan_ID               0
Gender                0
Married               3
Dependents            0
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [17]:
data.Married.value_counts()

Yes    398
No     213
Name: Married, dtype: int64

In [18]:
data.Married.fillna("Yes", inplace=True)

In [19]:
data.Self_Employed.value_counts()

No     500
Yes     82
Name: Self_Employed, dtype: int64

In [20]:
pd.crosstab(data.Self_Employed,data.Dependents)

Dependents,0,1,2,3+
Self_Employed,,,,
No,302,76,80,42
Yes,39,20,16,7


In [21]:
pd.crosstab(data.Self_Employed,data.Married)

Married,No,Yes
Self_Employed,,
No,171,329
Yes,28,54


In [22]:
pd.crosstab(data.Self_Employed,data.Gender)

Gender,Female,Male
Self_Employed,,
No,89,411
Yes,15,67


In [23]:
data[data.Self_Employed.isnull()].head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
11,LID001027,Male,Yes,2,Graduate,NaN,2500,1840.0,109.0,360.0,1.0,Urban,Y
19,LID001041,Male,Yes,0,Graduate,NaN,2600,3500.0,115.0,NaN,1.0,Urban,Y
24,LID001052,Male,Yes,1,Graduate,NaN,3717,2925.0,151.0,360.0,NaN,Semiurban,N
29,LID001087,Female,No,2,Graduate,NaN,3750,2083.0,120.0,360.0,1.0,Semiurban,Y
30,LID001091,Male,Yes,1,Graduate,NaN,4166,3369.0,201.0,360.0,NaN,Urban,N


In [24]:
data.Self_Employed.fillna("No", inplace=True)

In [25]:
data.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [26]:
data.groupby('Gender')['LoanAmount'].median()

Gender
Female    113.0
Male      130.0
Name: LoanAmount, dtype: float64

In [27]:
data.groupby('Self_Employed')['LoanAmount'].median()

Self_Employed
No     125.0
Yes    150.0
Name: LoanAmount, dtype: float64

In [28]:
data.LoanAmount.median()

128.0

In [29]:
median=data.groupby(['Self_Employed']).LoanAmount.transform('median')
median.head()

0    125.0
1    125.0
2    150.0
3    125.0
4    125.0
Name: LoanAmount, dtype: float64

In [30]:
data.LoanAmount.fillna(median, inplace=True)

In [31]:
data.Loan_Amount_Term.value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
36.0       2
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [32]:
data.Loan_Amount_Term.fillna(360, inplace=True)

In [33]:
data.Credit_History.value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [34]:
pd.crosstab(data.Credit_History,data.Education)

Education,Graduate,Not Graduate
Credit_History,,
0.0,63,26
1.0,380,95


In [35]:
data.Credit_History.fillna(1.0, inplace=True)

In [36]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [37]:
data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LID001002,Male,No,0,Graduate,No,5849,0.0,125.0,360.0,1.0,Urban,Y
1,LID001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LID001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LID001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LID001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LID001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,LID001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,LID001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,LID001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,LID001020,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [38]:
d1 = data.drop(labels = "Loan_ID", axis = 1)


In [39]:
d1.Education.head()

0        Graduate
1        Graduate
2        Graduate
3    Not Graduate
4        Graduate
Name: Education, dtype: object

In [40]:
d1.Gender=pd.get_dummies(d1.Gender)

In [41]:
d1.Married=pd.get_dummies(d1.Married)

In [42]:
d1.Education=pd.get_dummies(d1.Education)

In [43]:
d1.Self_Employed=pd.get_dummies(d1.Self_Employed)

In [44]:
d1.Loan_Status=pd.get_dummies(d1.Loan_Status)

In [45]:
d1.dtypes

Gender                 uint8
Married                uint8
Dependents            object
Education              uint8
Self_Employed          uint8
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status            uint8
dtype: object

In [46]:
d1.Property_Area.unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

In [47]:
def getname(name):
    title_group = {'Urban':0,'Rural':1,'Semiurban':2}  
    return title_group[name]
d1['Property_Area']=d1.Property_Area.map(lambda x : getname(x))

In [48]:
d1.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,1,1,5849,0.0,125.0,360.0,1.0,0,0
1,0,0,1,1,1,4583,1508.0,128.0,360.0,1.0,1,1
2,0,0,0,1,0,3000,0.0,66.0,360.0,1.0,0,0
3,0,0,0,0,1,2583,2358.0,120.0,360.0,1.0,0,0
4,0,1,0,1,1,6000,0.0,141.0,360.0,1.0,0,0


In [49]:
d1.dtypes

Gender                 uint8
Married                uint8
Dependents            object
Education              uint8
Self_Employed          uint8
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
Loan_Status            uint8
dtype: object

In [50]:
d1.Dependents.unique()

array([0, 1, 2, '3+'], dtype=object)

In [51]:
d1.Dependents.head()

0    0
1    1
2    0
3    0
4    0
Name: Dependents, dtype: object

In [52]:
d1.Loan_Amount_Term.unique()

array([360., 120., 240., 180.,  60., 300., 480.,  36.,  84.,  12.])

In [53]:
d1["Loan_Amount_Term"]= d1["Loan_Amount_Term"].map({360.: 0,120.: 1,240.: 2,180.: 3,60.:4,300.:5,480.:6,36.:7,84.:8,12.:9})

In [54]:
d1["Dependents"]= d1["Dependents"].map({0: 0,1: 1,2: 2,'3+': 3})

In [55]:
d1.head(5)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,1,1,5849,0.0,125.0,0,1.0,0,0
1,0,0,1,1,1,4583,1508.0,128.0,0,1.0,1,1
2,0,0,0,1,0,3000,0.0,66.0,0,1.0,0,0
3,0,0,0,0,1,2583,2358.0,120.0,0,1.0,0,0
4,0,1,0,1,1,6000,0.0,141.0,0,1.0,0,0


In [56]:
features = d1[["Gender","Married","Dependents","Education","Self_Employed","ApplicantIncome","CoapplicantIncome","LoanAmount","Loan_Amount_Term","Credit_History","Property_Area"]]

label = np.array(d1["Loan_Status"])

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=0)

In [58]:
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import zscore
from sklearn.preprocessing import Imputer
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [59]:
NNH = KNeighborsClassifier(n_neighbors=35,leaf_size=5, metric='minkowski',p=4)
NNH.fit(X_train,y_train)
predicted_labels = NNH.predict(X_test)
accuracy_score_knn3= accuracy_score(y_test, predicted_labels)
print(accuracy_score_knn3)


0.7317073170731707


In [ ]:
#2 z score one

In [60]:
z_data= features.apply(zscore)
z_data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,-0.472343,1.372089,-0.737806,0.528362,0.392601,0.072991,-0.554487,-0.247076,-0.371881,0.411733,-1.250298
1,-0.472343,-0.728816,0.253470,0.528362,0.392601,-0.134412,-0.038732,-0.211384,-0.371881,0.411733,-0.060092
2,-0.472343,-0.728816,-0.737806,0.528362,-2.547117,-0.393747,-0.554487,-0.949028,-0.371881,0.411733,-1.250298
3,-0.472343,-0.728816,-0.737806,-1.892641,0.392601,-0.462062,0.251980,-0.306564,-0.371881,0.411733,-1.250298
4,-0.472343,1.372089,-0.737806,0.528362,0.392601,0.097728,-0.554487,-0.056717,-0.371881,0.411733,-1.250298


In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(z_data, label, test_size=0.2, random_state=0)

In [62]:
NNH = KNeighborsClassifier(n_neighbors=35,leaf_size=5, metric='minkowski',p=4)
NNH.fit(X_train,y_train)
predicted_labels = NNH.predict(X_test)
accuracy_score_knn= accuracy_score(y_test, predicted_labels)
print(accuracy_score_knn)

0.8292682926829268


In [ ]:
#dt

In [70]:
DTClassifier_1= DecisionTreeClassifier(max_depth=4, min_samples_split = 3, min_samples_leaf=2)
DTClassifier_1.fit (X_train,y_train)
predicted_labels = DTClassifier_1.predict(X_test)
accuracy_score_1 = accuracy_score(y_test, predicted_labels)
print(accuracy_score_1)

0.8211382113821138
